In [3]:
import pandas as pd

df = pd.read_csv('exemplo3.csv')
df

,idade,conta_corrente,sexo,risco
0,21.257389,783.127911,masculino,ruim
1,21.766573,979.747521,feminino,bom
2,25.321033,1065.328054,feminino,bom
3,23.919268,1195.758078,feminino,bom
4,21.805298,1083.764450,feminino,bom
...,...,...,...,...
2495,63.840703,2847.204903,feminino,ruim
2496,57.367341,3238.149058,masculino,bom
2497,65.785408,2603.620751,masculino,ruim
2498,52.944838,2985.948214,masculino,ruim


## Dividir input e output

In [8]:
X = df.drop('risco', axis='columns')
y = df['risco']

## Transformando em binário os dados não numéricos

In [13]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler


X_cat = X.select_dtypes(include='object')

onehot = OneHotEncoder(sparse=False,drop="first")
X_bin = onehot.fit_transform(X_cat)
X_bin

array([[1.],
       [0.],
       [0.],
       ...,
       [1.],
       [1.],
       [1.]])

In [15]:
X_num = X.select_dtypes(exclude='object')
minmax = MinMaxScaler()
X_num = minmax.fit_transform(X_num)
X_num

array([[0.05586473, 0.06720224],
       [0.06499436, 0.09938945],
       [0.12872564, 0.11339923],
       ...,
       [0.85424954, 0.36522222],
       [0.62401886, 0.42781034],
       [0.61473291, 0.54031447]])

In [19]:
import numpy as np

X_all = np.append(X_num, X_bin, axis=1)
X_all

array([[0.05586473, 0.06720224, 1.        ],
       [0.06499436, 0.09938945, 0.        ],
       [0.12872564, 0.11339923, 0.        ],
       ...,
       [0.85424954, 0.36522222, 1.        ],
       [0.62401886, 0.42781034, 1.        ],
       [0.61473291, 0.54031447, 1.        ]])

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X_all, y, test_size=1/3, random_state=42)
svc = SVC()
svc.fit(X_train, y_train)
accuracy_score(y_test, svc.predict(X_test))

0.7146282973621103

## Testando um conjunto de dados novo

In [21]:
df_new = pd.DataFrame({'idade': [20, 25, 50], 'conta_corrente': [800, 4000, 2200], 'sexo': ['masculino', 'feminino', 'masculino']})
df_new

,idade,conta_corrente,sexo
0,20,800,masculino
1,25,4000,feminino
2,50,2200,masculino


In [28]:
X_new_bin = onehot.transform(df_new.select_dtypes(include='object'))
X_new_num = minmax.transform(df_new.select_dtypes(exclude='object'))
X_new = np.append(X_new_bin, X_new_num, axis=1)
svc.predict(X_new)

array(['ruim', 'bom', 'bom'], dtype=object)

In [29]:
df_previsao = df_new.copy()
df_previsao['previsao'] = svc.predict(X_new)
df_previsao

,idade,conta_corrente,sexo,previsao
0,20,800,masculino,ruim
1,25,4000,feminino,bom
2,50,2200,masculino,bom
